### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
df_rat = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\Python\\Notebook\\ml-latest-small_02\\ratings.csv', sep=',')

In [3]:
df_rat.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [67]:
# посчитаем средний рейтинг для каждого фильма
df_mean_rating = df_rat.groupby('movieId').mean().reset_index()[['movieId', 'rating']]
df_mean_rating.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [68]:
def classy(row):
    """Классифицирует фильмы по рейтингам: низкий, средний и высокий"""
    if row['rating'] <= 2:
        return 'Low'
    elif row['rating'] > 2 and row['rating'] <= 4:
        return 'Medium'
    elif row['rating'] > 4:
        return 'High'
    

In [69]:
# заполним столбец class
df_mean_rating['class'] = df_mean_rating.apply(classy, axis=1)
df_mean_rating.head()

,movieId,rating,class
0,1,3.872470,Medium
1,2,3.401869,Medium
2,3,3.161017,Medium
3,4,2.384615,Medium
4,5,3.267857,Medium


In [72]:
# проверим значения class
df_mean_rating['class'].value_counts()

Medium    6821
High      1272
Low        973
Name: class, dtype: int64

### Задание 2
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [43]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] 
}

In [47]:
df_keywords = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\Python\\Notebook\\ml-latest-small_02\\keywords.csv', sep=',', encoding='utf-8')

In [50]:
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [51]:
def geo_class(row):
    """Гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону."""
    if row['keyword'] in ['москва', 'тула', 'ярославль']:
        return 'Центр'
    elif row['keyword'] in ['петербург', 'псков', 'мурманск']:
        return 'Северо-Запад'
    elif row['keyword'] in ['владивосток', 'сахалин', 'хабаровск']:
        return 'Дальний Восток'
    else:
        return 'undefined'

In [56]:
# или так еще можно
def geo_class_1(row):
    """Гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону."""
    if row['keyword'] in geo_data['Центр']:
        return 'Центр'
    elif row['keyword'] in geo_data['Северо-Запад']:
        return 'Северо-Запад'
    elif row['keyword'] in geo_data['Дальний Восток']:
        return 'Дальний Восток'
    else:
        return 'undefined'

In [57]:
df_keywords['region'] = df_keywords.apply(geo_class_1, axis=1)
df_keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [58]:
df_keywords.region.value_counts()

undefined         99991
Центр                 3
Северо-Запад          3
Дальний Восток        3
Name: region, dtype: int64

### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

* для каждой строки пройдите по всем годам списка years
* если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
* если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
* Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [4]:
df_movies = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\Python\\Notebook\\ml-latest-small_02\\movies.csv', sep=',', encoding='utf-8')

In [5]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# соединим movies и ratings в один ДФ
df_merged = df_movies.merge(df_rat, how='left')
df_merged.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


In [7]:
# посчитаем средний рейтинг для каждого фильма
df = df_merged.groupby('title').mean().reset_index()[['title', 'rating']]
df.head()

,title,rating
0,"""Great Performances"" Cats (1998)",1.750000
1,$9.99 (2008),3.833333
2,'Hellboy': The Seeds of Creation (2004),2.000000
3,'Neath the Arizona Skies (1934),0.500000
4,'Round Midnight (1986),2.250000


In [14]:
# создаем список из годов через цикл
years = []
for year in range(1950, 2011):
    years.append(str(year))
    
years

['1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010']

In [15]:
def production_year(row):
    for year in years:
        if year in row['title']:
            return year
        else:
            return '1900'

In [16]:
# я не понимаю, как исправить эту ошибку?
df['year'] = df['title'].apply(production_year)
df.head()

TypeError: string indices must be integers

In [123]:
df.groupby('year').mean().sort_values('rating', ascending=False).head()

KeyError: 'year'